### Set up TensorFlow

In [1]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

2024-01-01 16:24:35.058162: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-01 16:24:35.058217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-01 16:24:35.059137: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-01 16:24:35.067552: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-01 16:24:35.798192: W tensorflow/compiler/tf2

TensorFlow version: 2.15.0


### Load a dataset

Load and prepare the MNIST dataset.  THe pixel values of the images range from 0 through 255.  Scale these values to a range of 0 to 1 by dividing the values by 255.0.  This also converts the sample data frmo integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### Build a machine learning model

Build a `tf.keras.Sequential` model:

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

2024-01-01 16:24:36.794436: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-01 16:24:36.818042: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-01 16:24:36.818087: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-01 16:24:36.819994: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-01 16:24:36.820078: I external/local_xla/xla/stream_executor

`Sequential` is useful for stacking layers where each layer has one input tensor and one output tensor.  Run the model and get predictions, which are a vector of logits, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

2024-01-01 16:24:37.756224: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


array([[-0.7793735 , -0.38966176,  0.16369823,  0.1446262 , -0.21706992,
        -0.3494459 , -0.47824484, -0.341223  ,  0.1302657 , -0.16983148]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class:

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.05531019, 0.08166854, 0.14202866, 0.13934554, 0.09705334,
        0.08501985, 0.07474526, 0.08572184, 0.13735878, 0.101748  ]],
      dtype=float32)

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example.  This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.
This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.math.log(1/10) ~= 2.3.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.4648705

Before you start training, configure and compile the model using Keras `Model.compile`. Set the `optimizer` class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [8]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

### Train and evaluate the model

Use the `Model.fit` method to adjust your model parameters and minimize the loss:

In [9]:
model.fit(x_train, y_train, epochs=5)

2024-01-01 16:32:32.244810: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2024-01-01 16:32:32.699930: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 1/5


2024-01-01 16:32:34.077291: I external/local_xla/xla/service/service.cc:168] XLA service 0x7efddd6e9d70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-01 16:32:34.077335: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2024-01-01 16:32:34.129483: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-01 16:32:34.223088: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1704155554.377039    6103 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 [==============================] - 9s 4ms/step - loss: 0.2908 - accuracy: 0.9150
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1436 - accuracy: 0.9568
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1056 - accuracy: 0.9688
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0876 - accuracy: 0.9732
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0765 - accuracy: 0.9761


The `Model.evaluate` method checks the model's performance, usually on a validation set or a test set.

In [10]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0709 - accuracy: 0.9777 - 896ms/epoch - 3ms/step


[0.07090403139591217, 0.9776999950408936]

If you want your model to return a probability, you can wrap the trained model, and attach a softmax to it:

In [11]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[6.3870424e-08, 4.1779845e-08, 8.8197652e-05, 1.0254742e-03,
        4.6889029e-12, 2.3802228e-07, 7.8159078e-14, 9.9887973e-01,
        1.4981215e-07, 6.0463981e-06],
       [1.6042733e-08, 6.3865673e-06, 9.9998820e-01, 3.0420167e-06,
        2.5258514e-15, 1.5780405e-06, 2.0908709e-08, 2.0016854e-13,
        6.6667650e-07, 4.9351869e-13],
       [2.5625829e-07, 9.9817657e-01, 5.5598866e-05, 2.5956055e-05,
        4.5595760e-05, 5.8512428e-06, 1.0392441e-05, 3.4313352e-04,
        1.3097523e-03, 2.6951293e-05],
       [9.9927217e-01, 4.4774885e-07, 5.1607436e-04, 2.5777572e-06,
        1.3379531e-06, 4.3906375e-06, 4.9109658e-05, 7.9027623e-06,
        1.3479312e-06, 1.4460791e-04],
       [8.9531659e-07, 3.1674322e-08, 1.1338793e-05, 1.6238904e-08,
        9.9867320e-01, 6.4156929e-08, 6.0781827e-07, 8.9433946e-05,
        4.2386401e-07, 1.2240136e-03]], dtype=float32)>